In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
def process_file(file_name):
  with open(file_name, "r") as file:
    text = file.read()
  return text

In [5]:
def choose_data(n):
  # Choose single movie (Kung Fu Panda) as data
  if n == 1:
    file_names = [ "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Family/kungfupanda_dialog.txt" ]
    data = process_file(file_names[0])
  # Choose 2 movies (Die Hard and Die Hard 2) as data
  if n == 2:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/diehard_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/diehard2_dialog.txt"]
    data = process_file(file_names[0]) + ' STARTTOKEN ENDTOKEN ' + process_file(file_names[1])

  # Choose 3 movies (all Starwars) as data
  if n == 3:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarsanewhope_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarstheempirestrikesback_dialog.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/imsdb_scenes_dialogs_nov_2015/imsdb_scenes_dialogs_nov_2015/dialogs/Action/starwarsattackoftheclones_dialog.txt"]
    data = ""
    for file_name in file_names:
      data += ' STARTTOKEN '
      data += process_file(file_name)
      data += ' ENDTOKEN '


  # TRY OTHER DATASET... maybe Friday the 13th or Nightmare on Elm... any series with a ton of movies
  # Or Halloween
  if n == 4:
    file_names = ["/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Sorcerer s Stone_0241527.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Chamber of Secrets_0295297.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Prisoner of Azkaban_0304141.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Goblet of Fire_0330373.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Half Blood Prince_0417741.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Order of the Phoenix_0373889.txt",
                  "/content/gdrive/My Drive/Colab Notebooks/Movie Corpus Analysis/raw_texts/raw_texts/Harry Potter and the Deathly Hallows Part 1_0926084.txt"]
    data = ""
    for file_name in file_names:
      data += ' STARTTOKEN '
      data += process_file(file_name)
      data += ' ENDTOKEN '

  return data

In [6]:
data = choose_data(4)

In [7]:
import re

def remove_URLs_and_Numbers(text): 
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'\d+', '', text)
  return text

In [8]:
text = remove_URLs_and_Numbers(data)

In [9]:
tokens = word_tokenize(text)

In [10]:
print(tokens)
print(len(tokens))

['STARTTOKEN', 'HARRY', 'POTTER', 'AND', 'THE', 'SORCERER', '’', 'S', 'STONE', 'BR', 'eee', 'Screenplay', 'by', 'Steve', 'Kloves', 'Based', 'on', 'the', 'novel', 'by', 'J.K.', 'Rowling', 'Shooting', 'Draft', '//', 'Blue', 'Revision', '//', '-', 'Pink', 'Revision', '//', '.', 'Yellow', 'Revision', '//', 'Green', 'Revision', '//', 'Gold', 'Revision', '//', 'Buff', 'Revision', '//', '‘', 'Salmon', 'Revision', '//', 'Cherry', 'Revision', '//', 'Tan', 'Revision', ',', '//', "''", 'White', 'Revision', '//', "''", 'Blue', 'Revision', '//', '™', 'Pink', 'Revision', '.', '//', '™', 'Yellow', 'Revision', '//', "''", 'Green', 'Revision', '//', 'CO', 'Revision', 'nd', 'Yellow', '//', '-', 'OMITTED', 'SCS', '.', '-', '&', 'PAGES', '-', '-', 'EXT', '.', 'PRIVET', 'DRIVE', '-', 'NIGET', ':', 'Dark', 'at', 'this', 'hour', ',', 'except', 'for', 'the', 'STREET', 'LAMPS', 'that', 'dot', 'the', 'street', ',', 'spilling', 'deep', 'pools', 'of', 'light', 'upon', 'the', 'ground', '.', 'On', 'the', 'far', 'co

In [11]:
tokens.insert(0, '<s>')
i=0
while i < len(tokens):
  if tokens[i] == '.':
    tokens[i] = '</s>'
    tokens.insert(i+1, '<s>')
  i += 1

In [12]:
print(tokens)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
from nltk.tokenize.sonority_sequencing import punctuation
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer

def remove_Stopwords_and_Punctuation_and_Stem(tokens):
  stemmer = SnowballStemmer(language='english')
  englishStopwords = stopwords.words('english')
  puncuation = string.punctuation
  # stemmer.stem(token)
  tokens = [ token.lower() for token in tokens if token not in englishStopwords and token not in punctuation ]
  return tokens

In [14]:
tokens = remove_Stopwords_and_Punctuation_and_Stem(tokens)

In [15]:
print(len(set(tokens)))

13155


In [16]:
lookup = {}
start_words = []

for i in range(len(tokens)-2):
  context = tokens[i] + ' ' + tokens[i+1]
  nextToken = tokens[i+2]
  if context not in lookup.keys():
    lookup[context] = {}
  if nextToken not in lookup[context].keys():
    lookup[context][nextToken] = 0
  if tokens[i] == '<s>':
    start_words.append(context)
  lookup[context][nextToken] += 1

In [17]:
print(len(lookup))

88041


In [18]:
for key in lookup.keys():
  count = 0
  for val in lookup[key]:
    count += lookup[key][val]
  for val in lookup[key]:
    lookup[key][val] /= count

In [19]:
print(lookup)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
import numpy as np
import numpy.random as rand

def generate_sentences(num_sentences, lookup=lookup):
  context = rand.choice(start_words)
  sentence = context
  current_token = None
  num_end_tokens = 0

  while num_end_tokens < num_sentences:

    probs = [ lookup[context][key] for key in lookup[context].keys() ]
    keys = [ key for key in lookup[context].keys() ]
    i = rand.multinomial(1,probs)
    
    index = 0
    for j in range(len(i)):
      if i[j] == 1:
        index = j
        break
    current_token = keys[index]
    context = context.split()[1] + ' ' + current_token
    sentence = sentence + ' ' + current_token
    if current_token == '</s>':
      num_end_tokens += 1
    
  return sentence

In [26]:
print(generate_sentences(10))

<s> harry looks windows es ceiling finds glorious stars icy blue moon </s> <s> harry just ignore ron ron huh ron looks hand finds face feels suddenly awkward </s> <s> mrs. weasley gives ron jab shoulder </s> <s> young </s> <s> it 's nephew </s> <s> harry ‘ messrs moony wormtail padfoot prongs proud present the marauder 's map lies open otherwise bare desk </s> <s> within ... plunges ... int </s> <s> pause it likely seeing quite lot harry potter and the deathly ... pt </s> <s> krum sits stoicly catching harry 's eye </s> <s> from within myrtle heard moaning </s>
